In [1]:
import sqlite3
conn = sqlite3.connect('Library.db')
cur = conn.cursor()
cur.execute('''
CREATE TABLE IF NOT EXISTS User (
    User_ID INTEGER PRIMARY KEY AUTOINCREMENT,
    User_Name TEXT NOT NULL UNIQUE,
    Password TEXT NOT NULL
)
''')
cur.execute('''
CREATE TABLE IF NOT EXISTS Book (
    Book_ID INTEGER PRIMARY KEY AUTOINCREMENT,
    Book_Title TEXT NOT NULL UNIQUE,
    Book_Author TEXT NOT NULL,
    Book_Price REAL NOT NULL,
    Quantity INTEGER NOT NULL
)
''')
cur.execute('''
CREATE TABLE IF NOT EXISTS IssueBook (
    Issue_ID INTEGER PRIMARY KEY AUTOINCREMENT,
    User_ID INTEGER,
    Book_ID INTEGER,
    Issue_Date TEXT,
    FOREIGN KEY(User_ID) REFERENCES User(User_ID),
    FOREIGN KEY(Book_ID) REFERENCES Book(Book_ID)
)
''')
cur.execute("SELECT COUNT(*) FROM Book")
if cur.fetchone()[0]==0:
    books=[
        ("Python Programming", "John Doe", 350.0, 3),
        ("Data Science 101", "Jane Smith", 500.0, 2),
        ("Deep Learning", "Andrew Ng", 750.0, 1)
    ]
    cur.executemany("INSERT INTO Book (Book_Title, Book_Author, Book_Price, Quantity) VALUES (?, ?, ?, ?)", books)
    conn.commit()

In [2]:
def register_user(user_name,password):
    try:
        cur.execute("INSERT INTO User (User_Name,Password) VALUES (?,?)", (user_name,password))
        conn.commit()
        print("Registration successful! You can now log in.")
    except sqlite3.IntegrityError:
        print("Username already exists.")

def login_user(user_name,password):
    cur.execute("SELECT User_ID FROM User WHERE User_Name = ? AND Password = ?", (user_name,password))
    user=cur.fetchone()
    return user[0] if user else None

In [3]:
def search_books(title):
    cur.execute("SELECT * FROM Book WHERE Book_Title LIKE ?", ('%' + title + '%',))
    return cur.fetchall()
    
def issue_book(user_id, book_id):
    cur.execute("SELECT Quantity FROM Book WHERE Book_ID = ?", (book_id,))
    result = cur.fetchone()

    if result and result[0] > 0:
        cur.execute("UPDATE Book SET Quantity = Quantity - 1 WHERE Book_ID = ?", (book_id,))
        issue_date =""
        cur.execute("INSERT INTO IssueBook (User_ID, Book_ID, Issue_Date) VALUES (?, ?, ?)",
                    (user_id, book_id, issue_date))
        conn.commit()
        print("Book issued successfully.")
    else:
        print("Book is not available.")

In [4]:
def Library():
    print("Welcome to Library Management System")
    print("1. Register\n2. Login")
    option = input("Select (1 or 2): ")
    if option == '1':
        user_name = input("Enter new username: ")
        password = input("Enter new password: ")
        register_user(user_name, password)
        
    user_name = input("\nUsername: ")
    password = input("Password: ")
    user_id = login_user(user_name, password)

    if user_id:
        print("Login successful!")

        print("\nAvailable Books:")
        cur.execute("SELECT * FROM Book WHERE Quantity > 0")
        available_books = cur.fetchall()

        if available_books:
            for book in available_books:
                print(f"ID: {book[0]} | Title: {book[1]} | Author: {book[2]} | ₹{book[3]} | In Stock: {book[4]}")
        else:
            print("No books currently available.")

        title = input("\nSearch for a book (or press Enter to skip): ")
        if title.strip():
            books = search_books(title)
            if books:
                print("\nMatching Books:")
                for book in books:
                    print(f"ID: {book[0]} | Title: {book[1]} | Author: {book[2]} | ₹{book[3]} | In Stock: {book[4]}")
            else:
                print("No matching books found.")
                return
        else:
            books = available_books

        book_id = input("\nEnter Book ID to issue (or press Enter to cancel): ")
        if book_id:
            issue_book(user_id, int(book_id))
    else:
        print("Invalid login.")

Library()
conn.close()

Welcome to Library Management System
1. Register
2. Login


Select (1 or 2):  1
Enter new username:  Tanveer
Enter new password:  pass123


Registration successful! You can now log in.



Username:  Tanveer
Password:  pass123


Login successful!

Available Books:
ID: 1 | Title: Python Programming | Author: John Doe | ₹350.0 | In Stock: 3
ID: 2 | Title: Data Science 101 | Author: Jane Smith | ₹500.0 | In Stock: 2
ID: 3 | Title: Deep Learning | Author: Andrew Ng | ₹750.0 | In Stock: 1



Search for a book (or press Enter to skip):  

Enter Book ID to issue (or press Enter to cancel):  1


Book issued successfully.
